<a href="https://colab.research.google.com/github/chalescharli/sentimental_analysist/blob/main/Sentiments_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
!pip install nest_asyncio


In [35]:
!pip install alpaca-trade-api transformers


In [36]:
import asyncio
from alpaca_trade_api import REST, Stream
from transformers import pipeline

In [37]:
API_KEY = '--------'
API_SECRET = '---------'
BASE_URL = 'https://paper-api.alpaca.markets/v2'

In [38]:
rest_client = REST(API_KEY, API_SECRET, BASE_URL)

In [39]:
async def news_data_handler(news):
    """Handle incoming news data."""
    # Combine headline and summary for analysis
    summary = news.summary or ""
    headline = news.headline or ""
    relevant_text = summary + " " + headline

    # Perform sentiment analysis
    sentiment = classifier(relevant_text)

    print(f"Processed News: {headline}")
    print(f"Sentiment: {sentiment}")

    # Trading logic based on sentiment
    if sentiment[0]['label'] == 'POSITIVE' and sentiment[0]['score'] > 0.95:
        # Place a buy order for 100 shares of AAPL
        print("Placing a buy order for 100 shares of AAPL.")
        rest_client.submit_order(
            symbol="AAPL",
            qty=100,
            side='buy',
            type='market',
            time_in_force='gtc'
        )

    elif sentiment[0]['label'] == 'NEGATIVE' and sentiment[0]['score'] > 0.95:
        # Place a sell order for 100 shares of AAPL
        print("Placing a sell order for 100 shares of AAPL.")
        rest_client.submit_order(
            symbol="AAPL",
            qty=100,
            side='sell',
            type='market',
            time_in_force='gtc'
        )

In [40]:
# Create a streaming client
stream_client = Stream(API_KEY, API_SECRET)

In [41]:
# Subscribe to news data for AAPL
stream_client.subscribe_news(news_data_handler, "AAPL")

In [47]:
async def main():
    await stream_client.run()

In [59]:
if __name__ == "__main__":
    asyncio.get_event_loop().create_task(main())

ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-6' coro=<main() done, defined at <ipython-input-47-442dc8a9d39f>:1> exception=RuntimeError('asyncio.run() cannot be called from a running event loop')>
Traceback (most recent call last):
  File "<ipython-input-47-442dc8a9d39f>", line 2, in main
    await stream_client.run()
  File "/usr/local/lib/python3.10/dist-packages/alpaca_trade_api/stream.py", line 1016, in run
    asyncio.run(self._run_forever())
  File "/usr/lib/python3.10/asyncio/runners.py", line 33, in run
    raise RuntimeError(
RuntimeError: asyncio.run() cannot be called from a running event loop
/usr/lib/python3.10/asyncio/base_events.py:1894: RuntimeWarning: coroutine 'Stream._run_forever' was never awaited
  handle = self._ready.popleft()
